In [2]:
from transformers import pipeline

/home/ben/.pyenv/versions/3.11.3/envs/huggingface/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/ben/.pyenv/versions/3.11.3/envs/huggingface/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
classifier("stop that bad robot")

[{'label': 'NEGATIVE', 'score': 0.998948872089386}]

In [6]:
speech_recognizer = pipeline('automatic-speech-recognition')

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/ben/.pyenv/versions/3.11.3/envs/huggingface/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

In [10]:
text = speech_recognizer('./bad.wav')['text']

In [12]:
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.9997933506965637}]

In [17]:
import pyaudio
import numpy as np
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# Load the pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Initialize PyAudio
p = pyaudio.PyAudio()

# Microphone stream configuration
RATE = 44100
CHUNK = 1024

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("Listening...")

def transcribe(audio):
    inputs = processor(audio, sampling_rate=RATE, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription[0]

try:
    while True:
        data = stream.read(CHUNK)
        audio_data = np.frombuffer(data, dtype=np.int16).astype(np.float32) / 32768.0
        transcription = transcribe(audio_data)
        print(f"Transcription: {transcription}")

except KeyboardInterrupt:
    print("Stopping...")

finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Listening...


In [15]:
import pyaudio

In [16]:
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    info = p.get_device_info_by_index(i)
    print(f"Device {i}: {info['name']} - Sample Rate: {info['defaultSampleRate']}")
    
p.terminate()

Device 0: HDA NVidia: V3L6W (hw:0,3) - Sample Rate: 44100.0
Device 1: HDA NVidia: HDMI 1 (hw:0,7) - Sample Rate: 44100.0
Device 2: HDA NVidia: HDMI 2 (hw:0,8) - Sample Rate: 44100.0
Device 3: HDA NVidia: HDMI 3 (hw:0,9) - Sample Rate: 44100.0
Device 4: Emotiva Little Ego: USB Audio (hw:1,0) - Sample Rate: 44100.0
Device 5: HD-Audio Generic: ALCS1200A Analog (hw:2,0) - Sample Rate: 44100.0
Device 6: HD-Audio Generic: ALCS1200A Alt Analog (hw:2,2) - Sample Rate: 44100.0
Device 7: AUKEY MI-U2 USB Mic: Audio (hw:3,0) - Sample Rate: 44100.0
Device 8: hdmi - Sample Rate: 44100.0
